In [1]:
from dataclasses import dataclass, field
from typing import TypedDict

In [52]:
# FIXME Turn 'Region' into a class and create an __add__ method
# or use recursion to explore path as done in previous challenge

@dataclass(slots=True, frozen=True)
class Plot:
    pid: str
    pos: tuple[int, int]
    borders: int

class Region(TypedDict):
    rid: str
    plots: set[Plot]
    area: int
    perimeter: int
    price: int

class PriceEstimator:
    def __init__(self, plots: list[Plot]):
        self.plots = plots
        self.regions = self._find_regions()
        self.region_starts = set()

    def _find_regions(self):
        self.regions = []
        
        for plot in self.plots:
            in_region = False
            
            for region in self.regions:
                if plot.pid == region["rid"][:len(plot.pid)] and self.is_in_region(plot, region):
                    region["plots"].add(plot)
                    region["area"] += 1
                    region["perimeter"] += plot.borders
                    in_region = True
                    break

            if not in_region:
                new_rid = self.get_new_rid(plot.pid)
                self.add_region(plot, new_rid)

        for region in self.regions:
            region["price"] = region["area"] * region["perimeter"]

        return self.regions

    def is_in_region(self, plot, region):
        for region_plot in region["plots"]:
            if self.are_adjacent(region_plot.pos, plot.pos) and region_plot.pid == plot.pid:
                return True
        return False
        
    def are_adjacent(self, pos1, pos2):
        x1, y1 = pos1
        x2, y2 = pos2
        return abs(x1 - x2) + abs(y1 - y2) == 1

    def get_new_rid(self, pid):
        counter = 1
        new_rid = f"{pid}{counter}"
        while any(region['rid'] == new_rid for region in self.regions):
            counter += 1
            new_rid = f"{pid}{counter}"
        return new_rid

    def add_region(self, plot, new_rid):
        region = {
            "rid": new_rid,
            "plots": {plot},
            "area": 1,
            "perimeter": plot.borders,
        }
        self.regions.append(region)

plots = []   

with open("data/test.txt", "r") as file:
    rows = [line.strip() for line in file]
    for x, row in enumerate(rows):
        print(row)
        for y, c in enumerate(row):
            borders = 0
            if (x > 0 and rows[x-1][y] != c) or x == 0:
                borders += 1
            if (x < len(rows) - 1 and rows[x+1][y] != c) or x == len(rows):
                borders += 1
            if (y > 0 and rows[x][y-1] != c) or y == 0:
                borders += 1
            if (y < len(row) - 1 and rows[x][y+1] != c) or y == len(row):
                borders += 1
                
            plots.append(Plot(c, (x, y), borders))

pe = PriceEstimator(plots)
print(*pe.regions, sep="\n\n")

RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE
{'rid': 'R1', 'plots': {Plot(pid='R', pos=(1, 0), borders=2), Plot(pid='R', pos=(0, 2), borders=1), Plot(pid='R', pos=(3, 2), borders=3), Plot(pid='R', pos=(1, 3), borders=1), Plot(pid='R', pos=(2, 2), borders=1), Plot(pid='R', pos=(1, 2), borders=0), Plot(pid='R', pos=(0, 0), borders=2), Plot(pid='R', pos=(2, 3), borders=1), Plot(pid='R', pos=(1, 1), borders=1), Plot(pid='R', pos=(0, 1), borders=1), Plot(pid='R', pos=(0, 3), borders=2), Plot(pid='R', pos=(2, 4), borders=3)}, 'area': 12, 'perimeter': 18, 'price': 216}

{'rid': 'I1', 'plots': {Plot(pid='I', pos=(0, 5), borders=2), Plot(pid='I', pos=(0, 4), borders=2), Plot(pid='I', pos=(1, 4), borders=2), Plot(pid='I', pos=(1, 5), borders=2)}, 'area': 4, 'perimeter': 8, 'price': 32}

{'rid': 'C1', 'plots': {Plot(pid='C', pos=(0, 6), borders=2), Plot(pid='C', pos=(0, 7), borders=2), Plot(pid='C', pos=(1, 8), borders=3), Plot(pid=